In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [130]:
df = pd.read_csv('train.csv')

In [131]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
y = df.label
X_full = df.drop(['label'],axis = 1,inplace=False)
X_full = X_full/255

X_dev = X_full.iloc[0:1000,:]
y_dev = y.iloc[0:1000]

X_train = X_full.iloc[1000:X_full.shape[0],:]
y_train = y.iloc[1000:y.shape[0]]



In [133]:
X_train.iloc[0]

pixel0      0.0
pixel1      0.0
pixel2      0.0
pixel3      0.0
pixel4      0.0
           ... 
pixel779    0.0
pixel780    0.0
pixel781    0.0
pixel782    0.0
pixel783    0.0
Name: 1000, Length: 784, dtype: float64

In [134]:
y_train.iloc[0]

1

## Part 1: Neural Network Implementation

### i)

In [135]:
def init_params(layer_sizes=[784,120,45,10]):
    num_layers = len(layer_sizes) - 1
    network = []
    for i in range(num_layers):
        weights = np.random.rand(layer_sizes[i+1],layer_sizes[i])
        biases = np.random.rand(layer_sizes[i+1],1)
        network.append([weights,biases])
    return network

In [136]:
network = init_params()
print(len(network))

3


### ii) Activation Functions

In [137]:
def ReLU(a):
    return np.maximum(0, a)

In [138]:
def softmax(z):
    z_max = np.max(z)
    z_exp = np.exp(z - z_max)
    sum_exp = np.sum(z_exp)
    softmax_z = np.round(z_exp / sum_exp, 3)
    return softmax_z

### iii) Forward Propagation

In [218]:
def forward_propagation(network,inp):
    pre_activation = [] ## Z values
    activation = [inp] ## A values, where inp is A0
    for i in range(len(network)):
        if(i!=len(network)-1):
            pre_activation.append(np.dot(network[i][0],activation[i]) + network[i][1])
            temp = ReLU(pre_activation[i]).reshape(-1,1)
            activation.append(temp)
            #print(pre_activation[i].shape,activation[i+1].shape)
        else:
            # last iteration which needs the softmax function.
            pre_activation.append(np.dot(network[i][0],activation[i]) + network[i][1])
            temp = softmax(pre_activation[i]).reshape(-1,1)
            activation.append(temp)
            #print(pre_activation[i].shape,activation[i+1].shape)
    return (pre_activation,activation)
    

In [225]:
# inp = np.random.rand(784,1)
inp = ((X_train.iloc[0,:]).to_numpy()).reshape(-1,1)
print(inp.shape)

pre_activation,activation = forward_propagation(network,inp)

print(activation[0].shape)
print(activation[1].shape)
print(activation[2].shape)
print(activation[3].shape)
print(pre_activation[0].shape)
print(pre_activation[1].shape)
print(pre_activation[2].shape)



(784, 1)
(784, 1)
(120, 1)
(45, 1)
(10, 1)
(120, 1)
(45, 1)
(10, 1)


### iv) One-Hot Encoding

In [235]:
def one_hot(Y):
    n_classes = 10  # We assume there are 10 possible classes (digits 0-9)
    try:
        m = len(Y)  # Number of examples
    except TypeError:
        m = 1       
    
    # Create an empty one-hot encoded array
    one_hot_Y = np.zeros((n_classes, m))
    
    # Assign 1 to the corresponding index for each element in Y
    try:
        for i, y in enumerate(Y):
            one_hot_Y[y, i] = 1
    except TypeError:
        one_hot_Y[Y] = 1
        return one_hot_Y
    return one_hot_Y.T

In [144]:
y_train.iloc[-1]

9

In [236]:
y_train.shape
one_hot(y_train)

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

## Part 2: Backward Propagation and Model Training

### i) Backward Propagation

In [237]:
def backward_propagation(pre_activation,activation,network,Y):
    pre_activation_gradients = []
    weight_gradients = []
    bias_gradients = []
    print(Y.shape)
    for i in reversed(range(len(network))):
        if(i!=len(network)-1):
            pre_activation_gradients.insert(0,np.multiply(np.dot(network[i+1][0].T, pre_activation_gradients[0]), np.int64(activation[i+1] > 0)))
            weight_gradients.insert(0,np.dot(pre_activation_gradients[0],activation[i].T))
            bias_gradients.insert(0,np.sum(pre_activation_gradients[0], axis=1, keepdims=True))
        else:
            ## for the end element, which needs to be dealt with first.
            pre_activation_gradients.insert(0,activation[i+1]-Y)
            print(pre_activation_gradients[0].shape)
            weight_gradients.insert(0,np.dot(pre_activation_gradients[0],activation[i].T))
            bias_gradients.insert(0,np.sum(pre_activation_gradients[0], axis=1, keepdims=True))
    return weight_gradients,bias_gradients

### ii) Update parameters

In [245]:
weight_gradients,bias_gradients = backward_propagation(pre_activation,activation,network,Y = one_hot(y_train.iloc[0]))
print(len(weight_gradients),len(bias_gradients))
print(weight_gradients[0].shape)
print(weight_gradients[1].shape)
print(weight_gradients[2].shape)
print(bias_gradients[0].shape)
print(bias_gradients[1].shape)
print(bias_gradients[2].shape)

(10, 1)
(10, 1)
3 3
(120, 784)
(45, 120)
(10, 45)
(120, 1)
(45, 1)
(10, 1)


In [271]:
def update_params(network,weight_gradients,bias_gradients,alpha):
    for layer in range(len(network)):
        network[layer][0] -= weight_gradients[layer]*alpha
        network[layer][1] -= bias_gradients[layer]*alpha
    return networkb